In [1]:
import pandas 
import pyhaystack
import json
import logging
logging.getLogger().setLevel(logging.INFO)

In [2]:
data = json.load(open("haystack-server.json","r")) # use_ssl = False or True depending on config
#Consider using pint = True due to return PintQuantity object for easier query
conn = pyhaystack.connect(**data)
op = conn.about()
op.wait()
res= op.result
print(res)

<Grid>
	Version: 3.0
	Columns:
		productUri
		tz
		moduleName
		serverName
		productName
		haystackVersion
		productVersion
		moduleVersion
		serverTime
		moduleUri
		serverBootTime
	Row    0:
	productUri=Uri('http://www.tridium.com/')
	tz='Sydney'
	moduleName='nhaystack'
	serverName='test'
	productName='Niagara 4'
	haystackVersion='2.0'
	productVersion='4.14.0.162'
	moduleVersion='3.3.0.0'
	serverTime=datetime.datetime(2025, 5, 9, 13, 41, 52, 14000, tzinfo=<DstTzInfo 'Australia/Sydney' AEST+10:00:00 STD>)
	moduleUri=Uri('https://github.com/ci-richard-mcelhinney/nhaystack')
	serverBootTime=datetime.datetime(2025, 5, 8, 14, 28, 39, 730000, tzinfo=<DstTzInfo 'Australia/Sydney' AEST+10:00:00 STD>)
</Grid>


In [3]:
op = conn.find_entity(filter_expr='energy')
op.wait()
site = op.result
print(list(site.keys()))
print("-----------------------------------------")
print(site)


['C.EMS.Meter_1', 'C.EMS.Meter_2', 'C.EMS.Meter_3']
-----------------------------------------
{'C.EMS.Meter_1': <@C.EMS.Meter_1: {actions='ver:"3.0"\ndis,expr\n"Override","pointOverride($self, $val, $duration)"\n"Auto","pointAuto($self)"\n"Emergency Override","pointEmergencyOverride($self, $val)"\n"Emergency Auto","pointEmergencyAuto($self)"\n"Set","pointSetDef($self, $val)"\n', axAnnotated, axSlotPath='slot:/EMS/Meter_1', axStatus='ok', axType='control:NumericWritable', cur, curErr='', curStatus='ok', curVal=PintQuantity(56666.0, <UnitsContainer({'kilowatt_hour': 1})>), dis='Meter_1', energy, his, hisErr='', hisInterpolate='linear', hisStatus='ok', kind='Number', maxVal=inf, minVal=-inf, n4SlotPath='slot:/EMS/Meter_1', navName='Meter_1', point, precision=1.0, tz='Sydney', unit='kWh', writable, writeErr='', writeLevel=10.0, writeStatus='ok', writeVal=56666.0}>, 'C.EMS.Meter_2': <@C.EMS.Meter_2: {actions='ver:"3.0"\ndis,expr\n"Override","pointOverride($self, $val, $duration)"\n"Auto","p

In [4]:
meter1 = site['C.EMS.Meter_1']
print(meter1)
print('\n --------------------------------')
print(meter1.tags)
print('----------------------------')
print(meter1.tags['curVal'])
print('---------------------')
print(meter1.tags['curVal'].value)
print(meter1.tags['curVal'].unit)

<@C.EMS.Meter_1: {actions='ver:"3.0"\ndis,expr\n"Override","pointOverride($self, $val, $duration)"\n"Auto","pointAuto($self)"\n"Emergency Override","pointEmergencyOverride($self, $val)"\n"Emergency Auto","pointEmergencyAuto($self)"\n"Set","pointSetDef($self, $val)"\n', axAnnotated, axSlotPath='slot:/EMS/Meter_1', axStatus='ok', axType='control:NumericWritable', cur, curErr='', curStatus='ok', curVal=PintQuantity(56666.0, <UnitsContainer({'kilowatt_hour': 1})>), dis='Meter_1', energy, his, hisErr='', hisInterpolate='linear', hisStatus='ok', kind='Number', maxVal=inf, minVal=-inf, n4SlotPath='slot:/EMS/Meter_1', navName='Meter_1', point, precision=1.0, tz='Sydney', unit='kWh', writable, writeErr='', writeLevel=10.0, writeStatus='ok', writeVal=56666.0}>

 --------------------------------
{actions='ver:"3.0"\ndis,expr\n"Override","pointOverride($self, $val, $duration)"\n"Auto","pointAuto($self)"\n"Emergency Override","pointEmergencyOverride($self, $val)"\n"Emergency Auto","pointEmergencyAu

In [5]:
for meter in list(site.keys()):
    values = site[meter]
    print(f"{meter} has {values.tags['curVal'].value} measured by {values.tags['curVal'].unit}")

C.EMS.Meter_1 has 56666.0 measured by kilowatt_hour
C.EMS.Meter_2 has 43752.0 measured by kilowatt_hour
C.EMS.Meter_3 has 43719.0 measured by kilowatt_hour


In [ ]:
#Get current data a stores to a database
import pandas as pd
import time

while True:
    op = conn.find_entity('energy')
    op.wait()
    meters = op.result
    for meter in list(meters.keys()):
        values = meters[meter]
        print(f"{meter} has {values.tags['curVal'].value} measured by {values.tags['curVal'].unit}")
    time.sleep(10)

C.EMS.Meter_1 has 55249.0 measured by kilowatt_hour
C.EMS.Meter_2 has 42335.0 measured by kilowatt_hour
C.EMS.Meter_3 has 42302.0 measured by kilowatt_hour
C.EMS.Meter_1 has 55259.0 measured by kilowatt_hour
C.EMS.Meter_2 has 42345.0 measured by kilowatt_hour
C.EMS.Meter_3 has 42312.0 measured by kilowatt_hour
C.EMS.Meter_1 has 55269.0 measured by kilowatt_hour
C.EMS.Meter_2 has 42355.0 measured by kilowatt_hour
C.EMS.Meter_3 has 42322.0 measured by kilowatt_hour
C.EMS.Meter_1 has 55278.0 measured by kilowatt_hour
C.EMS.Meter_2 has 42364.0 measured by kilowatt_hour
C.EMS.Meter_3 has 42331.0 measured by kilowatt_hour
C.EMS.Meter_1 has 55288.0 measured by kilowatt_hour
C.EMS.Meter_2 has 42374.0 measured by kilowatt_hour
C.EMS.Meter_3 has 42341.0 measured by kilowatt_hour


# Read history

In [6]:
#grid = conn.find_entity(filter_expr='his')
#print(grid) -> Only print out Meter 1 which is correct

his_id = conn.get_entity('energy', single=True)
his_id.wait()
test = his_id.result
print(test)

HaystackError: javax.baja.sys.BajaRuntimeException: Could not parse HRef 'energy'.

In [7]:
try:
    #grid = conn.read('point and sensor')
    print("Connection successfull. Data Received:")
    for row in grid:
        print(f"-> {row.dis}: {row['curVal']} {row.get('unit', '')}")
except Exception as e:
    print("Failed to connect or query data:")
    print(str(e))

Connection successfull. Data Received:
Failed to connect or query data:
'GetGridOperation' object is not iterable
